# SENG 474 A02: Assignment 2
## K-Fold Cross Validation
Sean McAuliffe, V00913346  
February 27, 2023


## Envrionmental Setup

In [12]:
# %pip install numpy
# %pip install pandas
# %pip install matplotlib
# %pip install seaborn
# %pip install scikit-learn
# %pip install scipy

import numpy as np
import matplotlib.pyplot as plt

## Data Import & Preprocessing 

In [13]:
from utils import mnist_reader
X_train, y_train = mnist_reader.load_mnist('data/fashion', kind='train')
X_test, y_test = mnist_reader.load_mnist('data/fashion', kind='t10k')

# Filter out examples which are not class 0 or 6
X_train = X_train[(y_train == 0) | (y_train == 6)]
y_train = y_train[(y_train == 0) | (y_train == 6)]
X_test = X_test[(y_test == 0) | (y_test == 6)]
y_test = y_test[(y_test == 0) | (y_test == 6)]

# Normalization Option 1: rescale so the examples lie in range [0, 1]
X_train = X_train / 255
X_test = X_test / 255

# Normalization Option 2: ensure each feature has unit norm
# X_train = X_train / np.linalg.norm(X_train, axis=1, keepdims=True)
# X_test = X_test / np.linalg.norm(X_test, axis=1, keepdims=True)

## *k*-Fold CV Implementation

In [1]:
def kfold_cv(X, y, k, model):
    # Split the data into k folds
    X_folds = np.array_split(X, k)
    y_folds = np.array_split(y, k)

    accuracies = []
    X_train = np.array([])
    y_train = np.array([])

    for i in range(k):
        # Create training and validation sets
        for j in range(k):
            if j == i:
                X_validation = X_folds[j]
                y_validation = y_folds[j]
            else:
                X_train = np.concatenate(X_train, X_folds[j])
                y_train = np.concatenate(y_train, y_folds[j])

        # Train the model
        model.fit(X_train, y_train)

        # Evaluate the model
        accuracies.append(model.score(X_validation, y_validation))

    return np.average(np.array(accuracies))